In Vesu the health factor is a measure of how “safe” our loan position is. It’s calculated by comparing our adjusted collateral against our outstanding debt.

Health Factor=(Total Collateral×Liquidation Threshold)/Total Debt
 

Total Collateral: The total value of the assets you’ve deposited.
Liquidation Threshold: A percentage (expressed as a decimal) that represents how much of your collateral counts toward covering your debt in a liquidation scenario.
Total Debt: The total value of your borrowed funds.

In summary, the health factor in Vesu indicates how close our collateral is to being insufficient to cover our debt. Monitoring this metric helps our manage risk, as a value below 1 signals that our position is at risk of liquidation.

In [ ]:
def calculate_health_factor(total_collateral, liquidation_threshold, total_debt):
    """
    Calculate the health factor ratio in Vesu.

    Parameters:
    - total_collateral (float): The total value of your collateral (e.g., in USD).
    - liquidation_threshold (float): The liquidation threshold as a decimal (e.g., 0.8 for 80%).
    - total_debt (float): The total value of your debt (e.g., in USD).

    Returns:
    - float: The health factor ratio. If total_debt is 0, returns float('inf').
    """
    if total_debt == 0:
        return float('inf')
    return (total_collateral * liquidation_threshold) / total_debt




The formula commonly used to calculate the dynamic interest rate in Vesu:

Utilization Rate (𝑢)

Calculation: 
𝑢=  Total Borrowed/ Total Liquidity 

Interest Rate ( 𝑟) 

Calculation: 
𝑟=   { 𝑟 0 + 𝑚 × 𝑢                     , if  𝑢 ≤ 𝑘 
      𝑟 0 + 𝑚 × 𝑘 + 𝑗 × ( 𝑢 − 𝑘 ),      , if  𝑢>k ​ }

 ​

Where:

r0= Base Rate (minimum interest rate)
m = Multiplier (rate increase per unit utilization below the kink)
k = Kink Point (the utilization threshold where the rate slope changes)
j = Jump Multiplier (steeper rate increase for utilization beyond the kink)

In [ ]:
def calculate_interest_rate(total_borrowed, total_liquidity, base_rate, multiplier, kink, jump_multiplier):
    """
    Calculate the dynamic interest rate based on pool utilization.

    Parameters:
    - total_borrowed (float): Total amount borrowed from the pool.
    - total_liquidity (float): Total deposits (liquidity) available in the pool.
    - base_rate (float): The base interest rate (e.g., 0.02 for 2% per year).
    - multiplier (float): The rate increase per unit of utilization below the kink.
    - kink (float): The utilization threshold (0 to 1) where the rate slope changes.
    - jump_multiplier (float): The steeper rate increase beyond the kink.

    Returns:
    - float: The computed interest rate.
    """
    if total_liquidity == 0:
        return base_rate  
    utilization_rate = total_borrowed / total_liquidity

    if utilization_rate <= kink:
        interest_rate = base_rate + utilization_rate * multiplier
    else:
        interest_rate = base_rate + kink * multiplier + (utilization_rate - kink) * jump_multiplier

    return interest_rate





In [4]:
!pip install nest_asyncio


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [10]:
import asyncio
import nest_asyncio
from decimal import Decimal
from typing import Dict, List, Tuple, Optional, Union
from IPython.display import display
from starknet_py.hash.selector import get_selector_from_name
from starknet_py.net.client_models import Call
from starknet_py.net.full_node_client import FullNodeClient

class StarknetClient:
    """
    A client class for interacting with the Starknet blockchain.
    Provides methods for making contract calls and fetching specific contract data.
    """
    def __init__(self, node_url: str = "https://starknet-mainnet.public.blastapi.io"):
        """
        Initialize StarknetClient with network URL.
        Args:
            node_url (str): The URL of the Starknet network to connect to
        """
        self.node_url = node_url
        self.client = FullNodeClient(node_url=self.node_url)
    
    async def func_call(
        self, addr: int, selector: str, calldata: List[int], retries: int = 1
    ) -> List[int]:
        """
        Make a call to a Starknet contract.
        Args:
            addr (int): Contract address
            selector (str): Function name to call
            calldata (List[int]): List of call arguments
            retries (int, optional): Number of retry attempts. Defaults to 1.
        Returns:
            List[int]: Contract call result
        """
        call = Call(
            to_addr=addr,
            selector=get_selector_from_name(selector),
            calldata=calldata,
        )
        for attempt in range(retries + 1):
            try:
                response = await self.client.call_contract(call, block_hash="latest")
                if hasattr(response, "result"):
                    return response.result
                elif isinstance(response, list):
                    return response
                else:
                    raise ValueError(f"Unexpected response type: {type(response)}")
            except Exception as e:
                if attempt == retries:
                    raise e
                await asyncio.sleep(10)
    
    async def balance_of(self, token_address: int, account_address: int) -> Decimal:
        """
        Get token balance for an account.
        Args:
            token_address (int): Address of the token contract
            account_address (int): Address of the account to check
        Returns:
            Decimal: Token balance
        """
        result = await self.func_call(token_address, "balanceOf", [account_address])
        return Decimal(result[0]) if result else Decimal(0)
    
    async def get_myswap_pool(
        self,
        pool_address: int,
        token_a_address: Optional[int] = None,
        token_b_address: Optional[int] = None,
    ) -> dict:
        """
        Get MySwap pool information.
        Args:
            pool_address (int): Address of the pool contract
            token_a_address (Optional[int]): Address of token A
            token_b_address (Optional[int]): Address of token B
        """
        reserves = await self.func_call(pool_address, "get_reserves", [])
        if not (token_a_address and token_b_address):
            token_a = await self.func_call(pool_address, "token0", [])
            token_b = await self.func_call(pool_address, "token1", [])
            token_a_address = token_a[0]
            token_b_address = token_b[0]
        return {
            "address": pool_address,
            "token_a": token_a_address,
            "token_b": token_b_address,
            "reserve_a": reserves[0],
            "reserve_b": reserves[1],
        }

"""Vesu contract address """
VESU_CONTRACT = 0x02545b2e5d519fc230e9cd781046d3a64e092114f07e44771e0d719d148725ef

"""Common tokens on Starknet """
ETH_ADDRESS = 0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7
USDC_ADDRESS = 0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8
DAI_ADDRESS = 0x00da114221cb83fa859dbdb4c44beeaa0bb37c7537ad5ae66fe5e0efd20e6eb3

"""User wallet address """
USER_ADDRESS = 0x0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef

async def analyze_vesu_protocol():
    """Analyze Vesu Protocol for a specific user"""
    client = StarknetClient()
    
    print("Analyzing Vesu Protocol for user...")
    collateral_data = await get_user_collateral(client)
    borrow_data = await get_user_borrows(client)
    health_factor = await calculate_health_factor(client, collateral_data, borrow_data)
    borrowing_potential = await calculate_borrowing_potential(client, collateral_data, borrow_data)
    print("\n===== VESU PROTOCOL ANALYSIS =====")
    print(f"Current Health Factor: {health_factor:.2f}")
    print("\nCollateral Positions:")
    for token, data in collateral_data.items():
        print(f"  - {token}: {data['amount']:.4f} (${data['usd_value']:.2f})")
    print("\nBorrowed Positions:")
    for token, data in borrow_data.items():
        print(f"  - {token}: {data['amount']:.4f} (${data['usd_value']:.2f})")
    print("\nBorrowing Potential:")
    for token, amount in borrowing_potential.items():
        print(f"  - {token}: {amount:.4f}")
    print("================================")

async def get_user_collateral(client: StarknetClient) -> Dict[str, Dict]:
    """Fetch user's collateral deposits in Vesu protocol"""
    try:
        collateral_result = await client.func_call(
            VESU_CONTRACT, 
            "get_user_collateral", 
            [USER_ADDRESS]
        )
        collateral = {}
        i = 0
        while i < len(collateral_result):
            token_address = collateral_result[i]
            token_amount = collateral_result[i+1]
            token_symbol = await get_token_symbol(client, token_address)
            price_usd = await get_token_price_usd(client, token_address)
            collateral[token_symbol] = {
                "address": token_address,
                "amount": Decimal(token_amount) / Decimal(10**18),  # Adjust decimals as needed
                "usd_value": (Decimal(token_amount) / Decimal(10**18)) * price_usd
            }
            i += 2
            
        return collateral
    except Exception as e:
        print(f"Error fetching collateral: {e}")
        return {
            "ETH": {"address": ETH_ADDRESS, "amount": Decimal("2.5"), "usd_value": Decimal("5000")},
            "USDC": {"address": USDC_ADDRESS, "amount": Decimal("3000"), "usd_value": Decimal("3000")}
        }

async def get_user_borrows(client: StarknetClient) -> Dict[str, Dict]:
    """Fetch user's borrowed assets in Vesu protocol"""
    try:
        borrow_result = await client.func_call(
            VESU_CONTRACT, 
            "get_user_borrows", 
            [USER_ADDRESS]
        )
        borrows = {}
        i = 0
        while i < len(borrow_result):
            token_address = borrow_result[i]
            token_amount = borrow_result[i+1]
            token_symbol = await get_token_symbol(client, token_address)
            price_usd = await get_token_price_usd(client, token_address)
            borrows[token_symbol] = {
                "address": token_address,
                "amount": Decimal(token_amount) / Decimal(10**18), 
                "usd_value": (Decimal(token_amount) / Decimal(10**18)) * price_usd
            }
            i += 2
            
        return borrows
    except Exception as e:
        print(f"Error fetching borrows: {e}")
        return {
            "DAI": {"address": DAI_ADDRESS, "amount": Decimal("2000"), "usd_value": Decimal("2000")}
        }

async def calculate_health_factor(
    client: StarknetClient, 
    collateral_data: Dict, 
    borrow_data: Dict
) -> Decimal:
    """Calculate user's health factor"""
    try:
        total_adjusted_collateral = Decimal("0")
        for token, data in collateral_data.items():
            threshold_result = await client.func_call(
                VESU_CONTRACT,
                "get_liquidation_threshold",
                [data["address"]]
            )
            liquidation_threshold = Decimal(threshold_result[0]) / Decimal(10000)

            adjusted_value = data["usd_value"] * liquidation_threshold
            total_adjusted_collateral += adjusted_value

        total_borrowed = sum(data["usd_value"] for data in borrow_data.values())
        if total_borrowed == 0:
            return Decimal("999") 
        
        health_factor = total_adjusted_collateral / total_borrowed
        return health_factor
        
    except Exception as e:
        print(f"Error calculating health factor: {e}")
        
        return Decimal("1.75")

async def calculate_borrowing_potential(
    client: StarknetClient, 
    collateral_data: Dict, 
    borrow_data: Dict
) -> Dict[str, Decimal]:
    """Calculate remaining borrowing potential"""
    try:
        current_health_factor = await calculate_health_factor(client, collateral_data, borrow_data)
        
        # Get minimum required health factor (typically 1.0)
        min_health_factor = Decimal("1.0")
        
        # Calculate total adjusted collateral value
        total_adjusted_collateral = Decimal("0")
        for token, data in collateral_data.items():
            threshold_result = await client.func_call(
                VESU_CONTRACT,
                "get_liquidation_threshold",
                [data["address"]]
            )
            liquidation_threshold = Decimal(threshold_result[0]) / Decimal(10000)
            adjusted_value = data["usd_value"] * liquidation_threshold
            total_adjusted_collateral += adjusted_value
        
       
        total_borrowed = sum(data["usd_value"] for data in borrow_data.values())
        
        
        max_additional_borrow_usd = (total_adjusted_collateral / min_health_factor) - total_borrowed
        
      
        if max_additional_borrow_usd <= 0:
            return {"USD": Decimal("0")}
        
        
        eth_price = await get_token_price_usd(client, ETH_ADDRESS)
        usdc_price = await get_token_price_usd(client, USDC_ADDRESS)
        dai_price = await get_token_price_usd(client, DAI_ADDRESS)
        
        borrowing_potential = {
            "ETH": max_additional_borrow_usd / eth_price,
            "USDC": max_additional_borrow_usd / usdc_price,
            "DAI": max_additional_borrow_usd / dai_price
        }
        
        return borrowing_potential
        
    except Exception as e:
        print(f"Error calculating borrowing potential: {e}")
        
        return {
            "ETH": Decimal("1.25"),
            "USDC": Decimal("2500"),
            "DAI": Decimal("2500")
        }

async def get_token_symbol(client: StarknetClient, token_address: int) -> str:
    """Get token symbol from address"""
    try:
        symbol_result = await client.func_call(token_address, "symbol", [])
        
        return f"TOKEN_{symbol_result[0]}"
    except Exception as e:
        # Return placeholder based on known addresses
        if token_address == ETH_ADDRESS:
            return "ETH"
        elif token_address == USDC_ADDRESS:
            return "USDC"
        elif token_address == DAI_ADDRESS:
            return "DAI"
        else:
            return f"TOKEN_{token_address % 1000}"

async def get_token_price_usd(client: StarknetClient, token_address: int) -> Decimal:
    """Get token price in USD"""
    try:
        
        price_result = await client.func_call(
            VESU_CONTRACT,
            "get_asset_price",
            [token_address]
        )
        return Decimal(price_result[0]) / Decimal(10**8)  
    except Exception as e:
        
        if token_address == ETH_ADDRESS:
            return Decimal("2000")
        elif token_address == USDC_ADDRESS:
            return Decimal("1")
        elif token_address == DAI_ADDRESS:
            return Decimal("1")
        else:
            return Decimal("10")  



async def main():
    """Main function to run the analysis"""
    await analyze_vesu_protocol()



nest_asyncio.apply()
asyncio.run(main())

Analyzing Vesu Protocol for user...
Error fetching collateral: Client failed with code -32603. Message: Internal error. Data: {'error': 'Invalid message selector'}
Error fetching borrows: Client failed with code -32603. Message: Internal error. Data: {'error': 'Invalid message selector'}
Error calculating health factor: Client failed with code -32603. Message: Internal error. Data: {'error': 'Invalid message selector'}
Error calculating health factor: Client failed with code -32603. Message: Internal error. Data: {'error': 'Invalid message selector'}
Error calculating borrowing potential: Client failed with code -32603. Message: Internal error. Data: {'error': 'Invalid message selector'}

===== VESU PROTOCOL ANALYSIS =====
Current Health Factor: 1.75

Collateral Positions:
  - ETH: 2.5000 ($5000.00)
  - USDC: 3000.0000 ($3000.00)

Borrowed Positions:
  - DAI: 2000.0000 ($2000.00)

Borrowing Potential:
  - ETH: 1.2500
  - USDC: 2500.0000
  - DAI: 2500.0000


In [12]:
import asyncio
import nest_asyncio
from decimal import Decimal
from typing import Dict, List, Optional
from IPython.display import display
from starknet_py.hash.selector import get_selector_from_name
from starknet_py.net.client_models import Call
from starknet_py.net.full_node_client import FullNodeClient

class StarknetClient:
    """
    A client class for interacting with the Starknet blockchain.
    Provides methods for making contract calls and fetching specific contract data.
    """
    def __init__(self, node_url: str = "https://starknet-mainnet.public.blastapi.io"):
        """
        Initialize StarknetClient with network URL.
        Args:
            node_url (str): The URL of the Starknet network to connect to.
        """
        self.node_url = node_url
        self.client = FullNodeClient(node_url=self.node_url)
    
    async def func_call(
        self, addr: int, selector: str, calldata: List[int], retries: int = 1
    ) -> List[int]:
        """
        Make a call to a Starknet contract.
        Args:
            addr (int): Contract address.
            selector (str): Function name to call.
            calldata (List[int]): List of call arguments.
            retries (int, optional): Number of retry attempts. Defaults to 1.
        Returns:
            List[int]: Contract call result.
        """
        call = Call(
            to_addr=addr,
            selector=get_selector_from_name(selector),
            calldata=calldata,
        )
        for attempt in range(retries + 1):
            try:
                response = await self.client.call_contract(call, block_hash="latest")
                if hasattr(response, "result"):
                    return response.result
                elif isinstance(response, list):
                    return response
                else:
                    raise ValueError(f"Unexpected response type: {type(response)}")
            except Exception as e:
                if attempt == retries:
                    raise e
                await asyncio.sleep(10)
    
    async def balance_of(self, token_address: int, account_address: int) -> Decimal:
        """
        Get token balance for an account.
        Args:
            token_address (int): Address of the token contract.
            account_address (int): Address of the account to check.
        Returns:
            Decimal: Token balance.
        """
        result = await self.func_call(token_address, "balanceOf", [account_address])
        return Decimal(result[0]) if result else Decimal(0)
    
    async def get_myswap_pool(
        self,
        pool_address: int,
        token_a_address: Optional[int] = None,
        token_b_address: Optional[int] = None,
    ) -> dict:
        """
        Get MySwap pool information.
        Args:
            pool_address (int): Address of the pool contract.
            token_a_address (Optional[int]): Address of token A.
            token_b_address (Optional[int]): Address of token B.
        """
        reserves = await self.func_call(pool_address, "get_reserves", [])
        if not (token_a_address and token_b_address):
            token_a = await self.func_call(pool_address, "token0", [])
            token_b = await self.func_call(pool_address, "token1", [])
            token_a_address = token_a[0]
            token_b_address = token_b[0]
        return {
            "address": pool_address,
            "token_a": token_a_address,
            "token_b": token_b_address,
            "reserve_a": reserves[0],
            "reserve_b": reserves[1],
        }

# Vesu contract address
VESU_CONTRACT = 0x02545b2e5d519fc230e9cd781046d3a64e092114f07e44771e0d719d148725ef

# Common tokens on Starknet
ETH_ADDRESS = 0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7
USDC_ADDRESS = 0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8
DAI_ADDRESS = 0x00da114221cb83fa859dbdb4c44beeaa0bb37c7537ad5ae66fe5e0efd20e6eb3

# User wallet address
USER_ADDRESS = 0x0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef

async def analyze_vesu_protocol():
    """Analyze Vesu Protocol for a specific user."""
    client = StarknetClient()
    
    print("Analyzing Vesu Protocol for user...")
    collateral_data = await get_user_collateral(client)
    borrow_data = await get_user_borrows(client)
    health_factor = await calculate_health_factor(client, collateral_data, borrow_data)
    borrowing_potential = await calculate_borrowing_potential(client, collateral_data, borrow_data)
    print("\n===== VESU PROTOCOL ANALYSIS =====")
    print(f"Current Health Factor: {health_factor:.2f}")
    print("\nCollateral Positions:")
    for token, data in collateral_data.items():
        print(f"  - {token}: {data['amount']:.4f} (${data['usd_value']:.2f})")
    print("\nBorrowed Positions:")
    for token, data in borrow_data.items():
        print(f"  - {token}: {data['amount']:.4f} (${data['usd_value']:.2f})")
    print("\nBorrowing Potential:")
    for token, amount in borrowing_potential.items():
        print(f"  - {token}: {amount:.4f}")
    print("================================")

async def get_user_collateral(client: StarknetClient) -> Dict[str, Dict]:
    """Fetch user's collateral deposits in Vesu protocol."""
    collateral_result = await client.func_call(
        VESU_CONTRACT, 
        "get_user_collateral", 
        [USER_ADDRESS]
    )
    collateral = {}
    i = 0
    while i < len(collateral_result):
        token_address = collateral_result[i]
        token_amount = collateral_result[i+1]
        token_symbol = await get_token_symbol(client, token_address)
        price_usd = await get_token_price_usd(client, token_address)
        collateral[token_symbol] = {
            "address": token_address,
            "amount": Decimal(token_amount) / Decimal(10**18),  # Adjust decimals as needed
            "usd_value": (Decimal(token_amount) / Decimal(10**18)) * price_usd
        }
        i += 2
    return collateral

async def get_user_borrows(client: StarknetClient) -> Dict[str, Dict]:
    """Fetch user's borrowed assets in Vesu protocol."""
    borrow_result = await client.func_call(
        VESU_CONTRACT, 
        "get_user_borrows", 
        [USER_ADDRESS]
    )
    borrows = {}
    i = 0
    while i < len(borrow_result):
        token_address = borrow_result[i]
        token_amount = borrow_result[i+1]
        token_symbol = await get_token_symbol(client, token_address)
        price_usd = await get_token_price_usd(client, token_address)
        borrows[token_symbol] = {
            "address": token_address,
            "amount": Decimal(token_amount) / Decimal(10**18),
            "usd_value": (Decimal(token_amount) / Decimal(10**18)) * price_usd
        }
        i += 2
    return borrows

async def calculate_health_factor(
    client: StarknetClient, 
    collateral_data: Dict, 
    borrow_data: Dict
) -> Decimal:
    """Calculate user's health factor."""
    total_adjusted_collateral = Decimal("0")
    for token, data in collateral_data.items():
        threshold_result = await client.func_call(
            VESU_CONTRACT,
            "get_liquidation_threshold",
            [data["address"]]
        )
        liquidation_threshold = Decimal(threshold_result[0]) / Decimal(10000)
        adjusted_value = data["usd_value"] * liquidation_threshold
        total_adjusted_collateral += adjusted_value

    total_borrowed = sum(data["usd_value"] for data in borrow_data.values())
    if total_borrowed == 0:
        return Decimal("999")
    health_factor = total_adjusted_collateral / total_borrowed
    return health_factor
        
async def calculate_borrowing_potential(
    client: StarknetClient, 
    collateral_data: Dict, 
    borrow_data: Dict
) -> Dict[str, Decimal]:
    """Calculate remaining borrowing potential."""
    # Get minimum required health factor (typically 1.0)
    min_health_factor = Decimal("1.0")
        
    # Calculate total adjusted collateral value
    total_adjusted_collateral = Decimal("0")
    for token, data in collateral_data.items():
        threshold_result = await client.func_call(
            VESU_CONTRACT,
            "get_liquidation_threshold",
            [data["address"]]
        )
        liquidation_threshold = Decimal(threshold_result[0]) / Decimal(10000)
        adjusted_value = data["usd_value"] * liquidation_threshold
        total_adjusted_collateral += adjusted_value
        
    total_borrowed = sum(data["usd_value"] for data in borrow_data.values())
    max_additional_borrow_usd = (total_adjusted_collateral / min_health_factor) - total_borrowed

    if max_additional_borrow_usd <= 0:
        return {"USD": Decimal("0")}
        
    eth_price = await get_token_price_usd(client, ETH_ADDRESS)
    usdc_price = await get_token_price_usd(client, USDC_ADDRESS)
    dai_price = await get_token_price_usd(client, DAI_ADDRESS)
        
    borrowing_potential = {
        "ETH": max_additional_borrow_usd / eth_price,
        "USDC": max_additional_borrow_usd / usdc_price,
        "DAI": max_additional_borrow_usd / dai_price
    }
    return borrowing_potential

async def get_token_symbol(client: StarknetClient, token_address: int) -> str:
    """Get token symbol from address."""
    symbol_result = await client.func_call(token_address, "symbol", [])
    return f"TOKEN_{symbol_result[0]}"

async def get_token_price_usd(client: StarknetClient, token_address: int) -> Decimal:
    """Get token price in USD."""
    price_result = await client.func_call(
        VESU_CONTRACT,
        "get_asset_price",
        [token_address]
    )
    return Decimal(price_result[0]) / Decimal(10**8)

async def main():
    """Main function to run the analysis."""
    await analyze_vesu_protocol()

nest_asyncio.apply()
asyncio.run(main())


Analyzing Vesu Protocol for user...


ClientError: Client failed with code -32603. Message: Internal error. Data: {'error': 'Invalid message selector'}